# Assignment 5:  MDPs and Q-learning On "Ice"

In this project we revisit Markov Decision Processes while also trying out Q-Learning, the reinforcement learning approach that associates utilities with attempting actions in states.
The problem that we’re attempting to solve is the following variation on the “Wumpus World” sometimes used as an example in lecture and the textbook.

1.  There is a grid of spaces in a rectangle.  Each space can contain a pit (negative reward), gold (positive reward), or nothing.
2.  The rectangle is effectively surrounded by walls, so anything that would move you outside the rectangle, instead moves you to the edge of the rectangle.
3.  The floor is icy.  Any attempt to move in a cardinal direction results in moving a somewhat random number of spaces in that direction.  The exact probabilities of moving each number of spaces are given in the problem description.  (If you slide too far, see rule #2.)
4.  Landing on a pit or gold effectively “ends the run,” for both a Q learner and an agent later trying out the policy.  It’s game over.  (To simulate this during Q learning, set all Q values for the space to equal its reward, then start over from a random space.)  Note that it's still possible to slide past a pit or gold - this doesn't end the run.

A sample input looks like this:


In [ ]:
sampleMDP = """0.7 0.2 0.1
- - P - -
- - G P -
- - P - -
- - - - -"""


The first line says that the probabilities of moving one, two, or three spaces in the direction of movement are 0.6, 0.3, and 0.1.   The rest is a map of the environment, where a dash is an empty space, P is a pit, and G is gold.

1.  mdp_solve() uses value iteration and the Bellman equation.  ITERATIONS refers to the number of complete passes you perform over all states.  I initialize the utilities to the rewards of each state.  We don’t update the rewards spaces from their initial rewards; since they end the trial, they have no future utility.  We also don't update utilities in-place as we iterate through them, but create a fresh array of utilities with each pass, in order to avoid biasing moves in the directions that have already been updated.

2.  q_solve() runs ITERATIONS trials in which a learner starts in a random empty square and moves until it hits a pit or gold, in which case, the trial is over.  (If it was randomly dropped into gold or a pit, the trial is immediately over.)  The learner moves by deciding randomly whether to choose a random direction (with probability EXPLORE_PROB) or move according to the best Q-value of its current square (otherwise).  We simulate the results of the move on slippery ice to determine where the learner ended up - then apply the Q-learning equation given in lecture and the textbook. 

The fact that a trial ends immediately on finding gold or a pit means that we want to handle those spaces in a special way.  Normally Q values are updated on moving to the next state, but we won’t see any next state in these cases.  So, to handle this, when the agent discovers one of these rewards, we set all the Q values for that space to the associated reward before quitting the trial.  So, for example, if gold is worth 100 and it’s discovered in square x, Q(x,UP) = 100, Q(x,RIGHT) = 100, Q(x, DOWN) = 100, and Q(x, LEFT) = 100.  There’s no need to apply the rest of the Q update equation when the trial is ending, because that’s all about future rewards, and there’s no future when the trial is ending.  But now the spaces that can reach that space will evaluate themselves appropriately.  (Before being "discovered," the square should have no utility.)

We use the GOLD_REWARD, PIT_REWARD, LEARNING_RATE, and DISCOUNT_FACTOR constants at the top of the file.


In [ ]:
""" "MDPs on Ice - Assignment 5"""

import sys
import numpy
import random
import copy

GOLD_REWARD = 250.0
PIT_REWARD = -150.0
DISCOUNT_FACTOR = 0.8
EXPLORE_PROB = 0.2 # for Q-learning
LEARNING_RATE = 0.01
ITERATIONS = 100000
MAX_MOVES = 1000
MIN = -1000
ACTIONS = 4
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3
MOVES = ['U', 'R', 'D', 'L']

# Fixed random number generator seed for result reproducibility --
# don't use a random number generator besides this to match sol
random.seed(5100)

class Problem:
    """Represents the physical space, transition probabilities, reward locations, and approach

    ...in short, the info in the text file

    Attributes:
        move_probs (List[float]):  probabilities of going 1,2,3 spaces
        map (List[List(string)]]:  "-" (safe, empty space), "G" (gold), "P" (pit)

    String format consumed looks like
    0.7 0.2 0.1   [probability of going 1, 2, 3 spaces]
    - - - - - - P - - - -   [space-delimited map rows]
    - - G - - - - - P - -   [G is gold, P is pit]

    You can assume the maps are rectangular, although this isn't enforced
    by this constructor.
    """

    def __init__(self, probstring):
        """ Consume string formatted as above"""
        self.map = []
        for i, line in enumerate(probstring.splitlines()):
            if i == 0:
                self.move_probs = [float(s) for s in line.split()]
            else:
                self.map.append(line.split())

    def solve(self, iterations, use_q):
        """ Wrapper for MDP and Q solvers.

        Args:
            iterations (int):  Number of iterations (but these work differently for the two solvers)
            use_q (bool):  False means use MDP value iteration, true means use Q-learning
        Returns:
            A Policy, in either case (what to do in each square; see class below)
        """

        if use_q:
            return q_solve(self, iterations)
        return mdp_solve(self, iterations)

class Policy:
    """ Abstraction on the best action to perform in each state.

    This is a string list-of-lists map similar to the problem input, but a character gives the best
    action to take in each non-reward square (see MOVES constant at top of file).
    """

    def __init__(self, problem):
        """Args:

        problem (Problem):  The MDP problem this is a policy for
        """
        self.best_actions = copy.deepcopy(problem.map)

    def __str__(self):
        """Join the characters in the policy into one big space-separated, multline string"""
        return '\n{}\n'.format('\n'.join([' '.join(row) for row in self.best_actions]))

def roll_steps(move_probs, row, col, move, rows, cols):
    """Calculates the new coordinates that result from a move.

    Includes the "roll of the dice" for transition probabilities and checking arena boundaries.

    Helper for try_policy and q_solve - probably useful in your Q-learning implementation.

    Args:
        move_probs (List[float]):  Transition probabilities for the ice (from problem)
        row, col (int, int):  location of agent before moving
        move (string):  The direction of move as a MOVES character (not an int constant!)
        rows, cols (int, int):  number of rows and columns in the map

    Returns:
        new_row, new_col (int, int):  The new row and column after moving
    """
    displacement = 1
    total_prob = 0
    move_sample = random.random()
    for p, prob in enumerate(move_probs):
        total_prob += prob
        if move_sample <= total_prob:
            displacement = p+1
            break
    # Handle "slipping" into edge of map
    new_row = row
    new_col = col
    if not isinstance(move, str):
        print("Warning: roll_steps wants str for move, got a different type")
    if move == "U":
        new_row -= displacement
        if new_row < 0:
            new_row = 0
    elif move == "R":
        new_col += displacement
        if new_col >= cols:
            new_col = cols-1
    elif move == "D":
        new_row += displacement
        if new_row >= rows:
            new_row = rows-1
    elif move == "L":
        new_col -= displacement
        if new_col < 0:
            new_col = 0
    return new_row, new_col


def try_policy(policy, problem, iterations):
    """Returns average utility per move of the policy.

    Average utility is as measured by "iterations" random drops of an agent onto empty
    spaces, running until gold, pit, or time limit MAX_MOVES is reached

    Args:
        policy (Policy):  the policy the agent is following
        problem (Problem):  the environment description
        iterations (int):  the number of random trials to run
    """
    total_utility = 0
    total_moves = 0
    for _ in range(iterations):
        # Resample until we have an empty starting square
        while True:
            row = random.randrange(0, len(problem.map))
            col = random.randrange(0, len(problem.map[0]))
            if problem.map[row][col] == "-":
                break
        for moves in range(MAX_MOVES):
            total_moves += 1
            policy_rec = policy.best_actions[row][col]
            # Take the move - roll to see how far we go, bump into map edges as necessary
            row, col = roll_steps(problem.move_probs, row, col, policy_rec, \
                                  len(problem.map), len(problem.map[0]))
            if problem.map[row][col] == "G":
                total_utility += GOLD_REWARD
                break
            if problem.map[row][col] == "P":
                total_utility += PIT_REWARD
                break
    return total_utility / total_moves

def validate(cord, rows, cols):
  row = cord[0]
  col = cord[1]

  if row < 0:
    row = 0
  elif row >= rows:
    row = rows - 1
  
  if col < 0:
    col = 0
  elif col >= cols:
    col = cols - 1
  
  return (row, col)

def mdp_solve(problem, iterations):
    """ Perform value iteration for the given number of iterations on the MDP problem.

    Here, the squares with rewards can be initialized to the reward values, since value iteration
    assumes complete knowledge of the environment and its rewards.

    Args:
        problem (Problem):  description of the environment
        iterations (int):  number of complete passes over the utilities
    Returns:
        a Policy (though you may design this to return utilities as a second return value)
    """
    # Initialize some variables.
    policy = Policy(problem)
    rows = len(problem.map)
    cols = len(problem.map[1])
    prev = []

    # Iterate through the rows.
    for row in range(rows):
      prev.append([])
      # Iterate through the columns.
      for col in range(cols):
        # Update the board state based on what we know.
        if problem.map[row][col] == '-':
          prev[row].append(0)
        elif problem.map[row][col] == 'P':
          prev[row].append(PIT_REWARD)
        elif problem.map[row][col] == 'G':
          prev[row].append(GOLD_REWARD)

    # Iterate for number of iterations specified.
    for ii in range(iterations):
      utils = []
      # Iterate through the rows.
      for row in range(rows):
        utils.append([])
        # Iterate through the columns.
        for col in range(cols):
          if problem.map[row][col] != '=':
            utils[row].append(prev[row][col])
            continue
          
          max_utility = -1000
          
          # Simulate the specified moves.
          for move in MOVES:
            new_coords = []
            for x in range(len(problem.move_probs)):
              new_coords.append([row, col])

            # Handle the designated current move direction.
            if move == 'U':
              for x in range(len(new_coords)):
                new_coords[x][0] -= (x + 1)
            elif move == "R":
              for x in range(len(new_coords)):
                new_coords[x][1] += (x + 1)
            elif move == "D":
              for x in range(len(new_coords)):
                new_coords[x][0] += (x + 1)
            elif move == "L":
              for x in range(len(new_coords)):
                new_coords[x][1] -= (x + 1)

            # Do the math.
            for x in range(len(new_coords)):
              new_coords[x] = list(get_validated_coord(tuple(new_coords[x]), num_rows, num_cols))
            utility = 0
            for x in range(len(problem.move_probs)):
              utility += problem.move_probs[x] * prev[new_coords[x][0]][new_coords[x][1]]
            utility = utility * DISCOUNT_FACTOR
            
            # Make the move if it is right.
            if utility > max_utility:
              max_utility = utility
              policy.best_actions[row][col] = move 
          utils[row].append(max_utility)
      
      # Update for next iteration.
      prev = utils  
    
    return policy

def q_solve(problem, iterations):
    """q_solve:  Use Q-learning to find a good policy on an MDP problem.

    Each iteration corresponds to a random drop of the agent onto the map, followed by moving
    the agent until a reward is reached or MAX_MOVES moves have been made.  When an agent
    is sitting on a reward, update the utility of each move from the space to the reward value
    and end the iteration.  (For simplicity, the agent also does this if just dropped there.)
    The agent does not "know" reward locations in its Q-values before encountering the space
    and "discovering" the reward.

    Note that texts differ on when to pay attention to this reward - this code follows the
    convention of scoring rewards of the space you are moving *from*, plus discounted best q-value
    of where you landed.

    Assume epsilon-greedy exploration.  Leave reward letters as-is in the policy,
    to make it more readable.  You'll probably want a helper for the q-update,
    to test independently.

    Args:
        problem (Problem):  The environment
        iterations (int):  The number of runs from random start to reward encounter
    Returns:
        A Policy for the map
    """
    # Initialize needed variables.
    policy = Policy(problem)
    rows = len(problem.map)
    cols = len(problem.map[1])
    qvals = []

    # Iterate through the rows.
    for row in range(rows):
      qvals.append([])
      # Iterate through the columns.
      for col in range(cols):
        # Update the board state based on what we know.
        if problem.map[row][col] == '-':
          qvals[row].append({'U': 0,'R': 0,'D': 0,'L': 0})
        elif problem.map[row][col] == 'P':
          qvals[row].append({'U': PIT_REWARD,'R': PIT_REWARD,'D': PIT_REWARD,'L': PIT_REWARD})
        elif problem.map[row][col] == 'G':
          qvals[row].append({'U': GOLD_REWARD,'R': GOLD_REWARD,'D': GOLD_REWARD,'L': GOLD_REWARD})
    
    # Iterate throught he designated iteration amount.
    for ii in range(iterations):
      s = (random.randint(0, rows - 1), random.randint(0, cols - 1))
      if problem.map[s[0]][s[1]] != '-':
        continue
      
      moves = 0
      
      # Go until we land on a game ending square or hit the  max moves.
      while problem.map[s[0]][s[1]] == '-' and moves < MAX_MOVES:
        action = 'U'
        row = s[0]
        col = s[1]
        maxq = qvals[row][col]['U']
        for move in MOVES:
          if qvals[row][col][move] > maxq:
            action = move
            maxq = qvals[row][col][move]
        if random.random() < EXPLORE_PROB:
          action = MOVES[random.randint(0, 3)]
        
        # Updates.
        next_row, next_col = roll_steps(problem.move_probs, row, col, action, rows, cols)
        next_s = (next_row, next_col)
        nmq = MIN
        
        for move in MOVES:
          if qvals[next_row][next_col][move] > nmq:
            nmq = qvals[next_row][next_col][move]
        
        qvals[row][col][action] += LEARNING_RATE * (DISCOUNT_FACTOR * nmq - qvals[row][col][action])
        maxq = MIN
        
        for move in MOVES:
          if qvals[row][col][move] > maxq:
            maxq = qvals[row][col][move]
            policy.best_actions[row][col] = move
        
        s = (next_row, next_col)
        moves += 1
    
    return policy

In [ ]:
deterministic_test = """1.0
- - P - -
- - G P -
- - P - -
- - - - -"""

In [ ]:
# Notice that we counterintuitively are most likely to go 2 spaces here
very_slippy_test = """0.2 0.7 0.1
- - P - -
- - G P -
- - P - -
- - - - -"""

In [ ]:
big_test = """0.6 0.3 0.1
- P - G - P - - G -
P G - P - - - P - -
P P - P P - P - P -
P - - P P - - - - P
- - - - - - - - P G"""

In [ ]:
# MDP value iteration tests
print(Problem(deterministic_test).solve(ITERATIONS, False))


- - P - -
- - G P -
- - P - -
- - - - -



In [ ]:
print(Problem(sampleMDP).solve(ITERATIONS, False))


- - P - -
- - G P -
- - P - -
- - - - -



In [ ]:
print(Problem(very_slippy_test).solve(ITERATIONS, False))


- - P - -
- - G P -
- - P - -
- - - - -



In [ ]:
print(Problem(big_test).solve(ITERATIONS, False))


- P - G - P - - G -
P G - P - - - P - -
P P - P P - P - P -
P - - P P - - - - P
- - - - - - - - P G



In [ ]:
# Q-learning tests
# Set seed every time for consistent executions;
# comment out to get different random runs
random.seed(5100)
print(Problem(deterministic_test).solve(ITERATIONS, True))


R D P R D
R R G P D
R U P D L
R U L L L



In [ ]:
random.seed(5100)
print(Problem(sampleMDP).solve(ITERATIONS, True))


D D P R D
R R G P D
U U P D D
U U L L L



In [ ]:
random.seed(5100)
print(Problem(very_slippy_test).solve(ITERATIONS, True))


D L P R D
R L G P L
D D P R D
U L U R U



In [ ]:
random.seed(5100)
print(Problem(big_test).solve(ITERATIONS, True))


U P R G L P R R G L
P G U P U R U P U U
P P U P P D P D P U
P D U P P D D D L P
R R U L L L L L P G



Here are a few thought questions:

**1) In an earlier version of this code, students sometimes found that some bad luck with pits near the end of the Q-learning training could cause square (0,1) of sample MDP to believe that going right was a bad idea.  What parameter would be best to adjust in order to mitigate the effect of bad luck near the end of a run?  In what direction should this parameter be adjusted?**

Because the pit is right behind where the gold is, we need to put priority on going right in this situation because we will hit the gold before the pit, causing the end of the game. We therefore need to change the problem state parameter to describe the probability and reward of going right differently. Keep 1 square the same, but since the gold is 2 away, we can group 2 and 3's probabilities with 2's rewards because it could never get to three spaces away (the pit).

**2) The value iteration MDP solver updates all squares an equal number of times.  The Q-learner does not.  Which squares might we expect the Q-learner to update the most?**

Because the Q-learner only updates when it is sitting on a reward, and it moreover only updates the squares/moves that led to a successful procurement of the reward, we can assume that the squares that are going to get updated the most are the ones with a clear straight shot to gold. If the start is placed random, and all further moves are also random, then the farther away we start the less likely we are to acheive, so those squares will get updated much less. square that have a ~1/4 chance of ending with a reward will therefore have a great chance of getting updated, and squares that are able to reach those squares will be the next most likely to be updated.

**3) Suppose we change the state information so that, instead of knowing its coordinates on the map, the agent instead knows just the locations of all rewards in a 5x5 square with the agent at the square center.  Thus, at the start of every run, it may not know exactly where it is, but it knows what is in the vicinity.  It also does not know the transition model.  (Suppose also that the map itself is very big, making it potentially worthwhile for an agent to "realize where it is" in the map when it isn't near gold.)**

**a) We can't use our value iteration here.  Why?**

**b) How many states are possible, assuming the contents of the agent's own square don't matter?  Is a lookup table of Q-values feasible?  (Let's define "feasible" as "able to be stored in a gig or less of memory," assuming 64-bit values.)**

**c) Suppose we were to use the Widrow-Hoff rule to generalize over these states instead, using the 48 boolean features of "pit at (row,col)" and "gold at (row,col)".  (The coordinates here refer to the 5x5 square.)  This is obviously less expensive, memory-wise, but what's another advantage over using a lookup table for the Q-values?  What's a disadvantage?**

**d) Now consider a reasonable neural network applied to this problem (details left up to you), compared to Widrow-Hoff.  What's an advantage?  What's a disadvantage?**


a.) If we never know exactly where we are, but we know where nearby gold is, we will never really be able to correlate our moves to success. The point of iterations is not ONLY to give us the gold, but also to avoid pits, and if we are solely trying to go for gold, we will have incosistnent success each time because the pits will be different based on where we are in the map. Since we can't track our coordinates, we can't say where we are for certain.

b.) We would have a grid of 5x5 squares minus the square we are currently standing on, so 24. Each square can have 3 possible values, and those values have no effect on what the value is next to them. Therefore we have 3^24 possible game states to keep track of, or roughly 280 billion possible states. This is not feasible both due to time and storage. 

c.) This style is obviously much tamer memory-wise, but it's also faster. Although lookup tables are constant time complexity, we see by the answer to part c that it would take a decent amount of time to find each state. With Widrow-Hoff rules, we will be able to generalize and provide weights to only the options available to us in the given state, thus minimizing the time it take to lookup and giving us a good shot at making a good decision. This would also be a much faster way of training. One downside to this method is that it will not give as accurate of results as the previous solution, becuase it is using a best guess whereas the previous solution was literally looking up the exact solution. 

d.) One advantage of using a neural network is that after training on many different games (both successes and failures) our neural network (assuming it does its job) will draw positive and negative correlations to the given game states that may not be able to be represented by the simple weight system that Widrow-Hoff uses or apparent to a human. A major downside is that you have to be very careful what you traing the neural network on becuase if it attaches to a correlation that is not representative of the problem space as a whole, then it will start inaccurately assessing future situations and making poor decisions. 